<a href="https://colab.research.google.com/github/kgpark88/nlp/blob/main/pi_mask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re

def is_hangul(s):
    is_han = False
    # hangul = re.compile('[^ ㄱ-ㅣ가-힣+]') # 한글과 공백을 제외한 모든 글자
    hangul = re.compile('[^ㄱ-ㅣ가-힣+]') # 한글을 제외한 모든 글자
    result = hangul.sub('', s) # 한글을 제외한 모든 부분을 제거
    if result: 
        is_han = True
    return is_han


def all_mask(pi):
    """
    앞의 n개 문자를 "*"로 마스킹 하는 함수, 구분 문자("-")는 마스킹에서 제외
    :param no: 개인정보
    :return: Masked String
    """
    masked = '*'*len(pi)
    return masked

def front_mask(pi, count):
    """
    앞의 n개 문자를 "*"로 마스킹 하는 함수, 구분 문자("-")는 마스킹에서 제외
    :param no: 개인정보
    :param count: 마스크할 갯수
    :return: Masked String
    """
    masked = ''
    if(len(pi) >= count):
        c = '-'
        c_list = [pos for pos, char in enumerate(pi) if char == c]
        pi2 = pi.replace(c, '')   
        pi2_length = len(pi2)
        masked = pi2[:count] + "*"*(pi2_length-count)   
        for i in c_list:
            masked = masked[:i] + c + masked[i:]
    return masked


def back_mask(pi, count):
    """
    뒤의 n개 문자를 "*"로 마스킹 하는 함수, 구분 문자("-")는 마스킹에서 제외
    :param pi: 개인정보
    :param count: 마스크할 갯수
    :return: Masked String
    """
    masked = ''
    if(len(pi) >= count):
        c = '-'
        c_list = [pos for pos, char in enumerate(pi) if char == c]
        pi2 = pi.replace(c, '')
        masked = pi2[:-count] +  "*"*count
        for i in c_list:
            masked = masked[:i] + c + masked[i:]
    return masked


def card_number_mask(card_no):
    """
    5~12번째 숫자와 마지막 자리수를 "*"로 마스킹 하는 함수, 구분 문자("-")는 마스킹에서 제외
    :param pi: 카드번호
    :return: Masked String
    """
    masked = ''
    cn = card_no.split('-')
    if(len(cn) == 4):
        masked = f'{cn[0]}-****-****-{cn[-1][:-1]}*'
    return masked


def name_mask(name):
    """
    한국성명 : 이름 앞자리와 마지막자리 사이 마스킹
    외국성명 : 영문 앞 4자리를 제외한 나머지
    :param name: 이름
    :return: Masked String
    """
    masked = ''
    name = name.strip().replace(' ','')
    name_length = len(name)
    if is_hangul(name):
        if name_length == 1:
            masked = name
        elif name_length == 2:
            masked = "{}*".format(name[0])
        if name_length > 2:
            masked = "{}{}{}".format(name[0], "*"*(name_length-2), name[-1])
    else:
        if name_length >= 4:
            masked = "{}{}".format(name[:4], "*"*(name_length-4))
        else: 
            masked = name
    return masked


def phone_no_mask(phone_no):
    """
    전화번호 마스킹 : 국번 외 블록의 앞 2자리를 제외한 나머지 번호를 마스킹
    :no : 전화번호
    :return: Masked String
    """
    masked = ''
    no = phone_no.split('-')
    if(len(no) != 3): return ''
    masked = f'{no[0]}-{no[1][:2]}**-{no[2][:2]}**'
    return masked


def email_mask(email):
    """
    이메일 마스킹 : 아이디 앞 2자리를 제외한 나머지를 마스킹
    :email : 이메일 주소
    :return: Masked String
    """
    masked = ''
    lst = email.split('@')
    if(len(lst) != 2): 
        masked =  ''
    else:
        email_id = lst[0]
        domain_name = lst[1]
        email_id_length = len(email_id) 
        if email_id_length >= 2:
            masked = masked = "{}{}{}{}".format(email_id[:2], "*"*(email_id_length-2), "@", domain_name)
        else:
            masked = masked = "{}{}{}".format(email_id, "@", domain_name)
    return masked


def ip_mask(ip):
    """
    IP 주소 마스킹 : 버전 4는 17~24비트 영역, 버전 6sms 113~128 비트 영역을 마스킹
    :email : 이메일 주소
    :return: Masked String
    """
    masked = ''
    if '.' in ip:
        ip_list = ip.split('.')
        ip_list[2] = '*'*len(ip_list[2])
        masked = '.'.join(ip_list)
    if ':' in ip:
        ip_list = ip.split(':')
        ip_list[-1] = '*'*len(ip_list[-1])
        masked = ':'.join(ip_list)
    return masked


def address_mask(address):
    """
    주소 마스킹 : 주소 마지막 부분을 마스킹
    :date : 년월일(2020-01-01, 20-01-01, 2020년01월01일)
    :return: Masked String
    """
    masked = ''
    address_list = address.split(" ")
    jibun_list = []
    doro_list = []
    for i, s in enumerate(address_list):
        if s.endswith('군') or s.endswith('읍') or s.endswith('면') or s.endswith('동'):
             jibun_list.append(i) 
        if s.endswith('로'):
             doro_list.append(i) 
    if jibun_list:
        idx = max(jibun_list)       
        for i in range(idx+1, len(address_list)):
            address_list[i] = "*"*len(address_list[i])
        masked = " ".join(address_list) 
    if doro_list:
        idx = max(doro_list)       
        for i in range(idx+2, len(address_list)):
            address_list[i] = "*"*len(address_list[i])
        masked = " ".join(address_list) 
    return masked   


def date_mask(date):
    """
    날짜 마스킹 : 년월일의 각 뒤 2자리를 마스킹
    :date : 년월일(2020-01-01, 20-01-01, 2020년01월01일)
    :return: Masked String
    """
    masked = ''
    date_length = len(date)
    if date_length==11 and "년" in date :
        masked = "{}{}{}{}{}{}{}".format(date[0:2], '**', date[4:5], '**', date[7:8], '**', date[10:11])
    elif date_length==10  and "-" in date:
        masked = "{}{}{}{}{}{}".format(date[:2], '**', date[4:5], '**', date[7:8], '**')
    elif date_length==8:
        masked = "{}{}{}{}{}".format('**', date[2:3], '**', date[5:6], '**')
    return masked




In [2]:

# 개인(신용)정보 화면 마스킹 Level 1 사용법
print('-'*70)
print('L1-01 주민등록번호 마스킹(뒤에서 6자리) : ' , back_mask('654321-1234567', 6))
print('L1-02 운전번호 마스킹(뒤에서 4자리) : ' , back_mask('12-34-567890-00', 4))
print('L1-03 운전번호 일련번호 : ' , back_mask('P71234', 4))
print('L1-04 여권번호 : ' , back_mask('M7961234', 4))
print('L1-05 외국인등록번호 : ' , back_mask('654321-1234567', 4))
print('L1-06 계좌번호 : ' , back_mask('123-02-1234567', 5))
print('L1-06 계좌번호 : ' , back_mask('1234556-01-112345', 5))
print('L1-06 계좌번호(핸드폰형태) : ' , back_mask('01056551234', 5))
print('L1-07 카드번호 : ' , card_number_mask('1234-2345-3456-1234'))
print('L1-07 카드번호 : ' , card_number_mask('9430-2345-3456-2398'))


# 개인(신용)정보 화면 마스킹 Level 2 사용법
print('-'*70)
print('L2-01 한국성명 : ' , name_mask('홍길동'))
print('L2-01 한국성명 : ' , name_mask('선남선녀'))
print('L2-01 한국성명 : ' , name_mask('연개소문'))
print('L2-01 한국성명 : ' , name_mask('이강'))
print('L2-01 외국성명 : ' , name_mask('hong gil dong'))
print('L2-02 핸드폰번호 : ', phone_no_mask('010-1234-1234'))
print('L2-03 전화번호 : ' , phone_no_mask('02-1234-1234'))
print('L2-04 전화번호 : ' , phone_no_mask('052-1234-1234'))
print('L2-04 이메일 : ' , email_mask('abcdefg@naver.com'))
print('L2-04 이메일 : ' , email_mask('Cd12345@goldwing.shinhan.com'))
print('L2-05 미국납세자번호 : ' , back_mask('912-34-1234', 4))
print('L2-06 차량번호 : ' , back_mask('12가1234', 4))
print('L2-07 IMEI, UUID등 단말기 고유식별번호 : ' , back_mask('358123456123456', 6))
print('L2-08 SNS주소 : ' , front_mask('a3123456789', 2))
print('L2-09 국내거소신고번호 : ' , back_mask('654321-1234567', 6))
print('L2-10 개인통관번호 : ' , front_mask('P812123456789', 4))
print('L2-11 본인확인기관이 부여한 고유번호(CI/아이핀번호 : ' , front_mask('cDR34-cw123456789', 7))
print('L2-12 ID : ' , front_mask('abc12345', 3))
print('L2-12 ID : ' , front_mask('f2g1234', 3))
print('L2-13 타사가 부여한 외부식별번호 : ' , front_mask('52123456', 2))
print('L2-13 타사가 부여한 외부식별번호 : ' , front_mask('951234', 2))

# 개인(신용)정보 화면 마스킹 Level 3 사용법
print('-'*70)
print('L3-01 내용_CLOB   : ' , front_mask('계약자가 전화를 안 받아서....', 10))
print('L3-02 내용_VC1000 : ' , front_mask('계약자가 전화를 안 받아서....', 10))
print('L3-03 내용_VC4000 : ' , front_mask('계약자가 전화를 안 받아서....', 10))
print('L3-04 입력 값 형태가 정해지지 않은 데이터(CLOB, VARCHAR 등) : ' , front_mask('계약자가 전화를 안 받아서....', 10))


# 개인(신용)정보 화면 마스킹 Level 3 사용법
print('-'*70)
print('L4-01 IP : ' , ip_mask('123.123.123.123'))
print('L4-01 IP : ' , ip_mask('1324:1234:abcd:ffff:0000:0000FFFF:1234'))
print('L4-02 주소 (지번) 상세주소 영역(읍/면/동 미만) : ' , address_mask('서울시 중구 순화동 AI빌딩'))
print('L4-02 주소 (도로면 상세주소 영역(도로명 미만): ' ,  address_mask('서울시 중구 세종대로 777길 AI빌딩'))
print('L4-02 주소 (도로면 상세주소 영역(도로명 미만): ' ,  address_mask('서울시 중구 세종대로 777 AI빌딩'))
print('L4-03 생년월일 : ' ,  date_mask('2020-01-01'))
print('L4-03 생년월일 : ' ,  date_mask('20-01-01'))
print('L4-03 생년월일 : ' ,  date_mask('2020년01월10일'))
print('L4-04 그룹사 고객번호 : ' ,  front_mask('123456789', 2))
print('L4-05 타사 ID(그룹통합/SK 등 : ' ,  front_mask('ab1234', 2))
print('L4-06 타사 계약번호 : ' ,  front_mask('12345678', 2))
print('L4-07 타사 고객번호 : ' ,  front_mask('12345678', 2))
print('L4-08 군번 : ' ,  front_mask('20-1234', 2))
print('L4-09 사건번호 : ' ,  front_mask('2012-123-1234567', 2))
print('L4-10 은행지로번호 : ' ,  front_mask('123456789', 2))
print('L4-11 생명보험협회ID : ' ,  front_mask('123456789', 2))
print('L4-12 직장명, 부서명, 직급, 국가, 성별, 키, 체중, 혼인여부, 재산, 가족정보 등 : ' ,  all_mask('ABC컴퍼니'))

----------------------------------------------------------------------
L1-01 주민등록번호 마스킹(뒤에서 6자리) :  654321-1******
L1-02 운전번호 마스킹(뒤에서 4자리) :  12-34-5678**-**
L1-03 운전번호 일련번호 :  P7****
L1-04 여권번호 :  M796****
L1-05 외국인등록번호 :  654321-123****
L1-06 계좌번호 :  123-02-12*****
L1-06 계좌번호 :  1234556-01-1*****
L1-06 계좌번호(핸드폰형태) :  010565*****
L1-07 카드번호 :  1234-****-****-123*
L1-07 카드번호 :  9430-****-****-239*
----------------------------------------------------------------------
L2-01 한국성명 :  홍*동
L2-01 한국성명 :  선**녀
L2-01 한국성명 :  연**문
L2-01 한국성명 :  이*
L2-01 외국성명 :  hong*******
L2-02 핸드폰번호 :  010-12**-12**
L2-03 전화번호 :  02-12**-12**
L2-04 전화번호 :  052-12**-12**
L2-04 이메일 :  ab*****@naver.com
L2-04 이메일 :  Cd*****@goldwing.shinhan.com
L2-05 미국납세자번호 :  912-34-****
L2-06 차량번호 :  12가****
L2-07 IMEI, UUID등 단말기 고유식별번호 :  358123456******
L2-08 SNS주소 :  a3*********
L2-09 국내거소신고번호 :  654321-1******
L2-10 개인통관번호 :  P812*********
L2-11 본인확인기관이 부여한 고유번호(CI/아이핀번호 :  cDR34-cw*********
L2-12 ID :  abc*****
L2-12 ID 